In [23]:
import pandas as pd
import numpy as np
from os import walk

In [2]:
f = []
for (dirpath, dirnames, filenames) in walk("../data/entity_recognition_datasets/"):
    f.extend(filenames)
    break

In [7]:
f

['32. Distribution Efficiency Data.csv',
 '6.MS_data_for_demo.csv',
 '25. Shipment Invoice Report.xlsx',
 '15. Bank - Loan Disbursment.xlsx',
 '31. Spends and Consumption Analytics.XLSX',
 '6. Market Intelligence - Competitor Performance.xlsx',
 '5. Plant Demand Fulfilment.xlsx',
 '2. Pharma - Secondary Sales.xlsx',
 '21. Travel and Expense Analysis Report.xlsx',
 '3. Regional Sales Performance.xlsx',
 '12. Insurance - Agent Scorecard.xlsx',
 '10. FMCG Delivery Performance.xlsx',
 '1. Pharma - Primary Sales.xlsx',
 '17. Bank - Talent Acquisition.csv']

In [15]:
data_sets = {}
column_names = []
data_name = []

for i, file_name in enumerate(f):
    print(file_name)
    if file_name.split(".")[2] == "csv":
        data_sets[f"data_{i}"] = pd.read_csv(f"../data/entity_recognition_datasets/{file_name}")
        column_names.extend(data_sets[f"data_{i}"].columns.tolist())
        data_name.extend([file_name for i in range(len(data_sets[f"data_{i}"].columns.tolist()))])
    else:
        data_sets[f"data_{i}"] = pd.read_excel(f"../data/entity_recognition_datasets/{file_name}", engine="openpyxl")
        column_names.extend(data_sets[f"data_{i}"].columns.tolist())
        data_name.extend([file_name for i in range(len(data_sets[f"data_{i}"].columns.tolist()))])



32. Distribution Efficiency Data.csv
6.MS_data_for_demo.csv
25. Shipment Invoice Report.xlsx
15. Bank - Loan Disbursment.xlsx
31. Spends and Consumption Analytics.XLSX
6. Market Intelligence - Competitor Performance.xlsx
5. Plant Demand Fulfilment.xlsx
2. Pharma - Secondary Sales.xlsx
21. Travel and Expense Analysis Report.xlsx
3. Regional Sales Performance.xlsx
12. Insurance - Agent Scorecard.xlsx
10. FMCG Delivery Performance.xlsx
1. Pharma - Primary Sales.xlsx
17. Bank - Talent Acquisition.csv


In [31]:
zipped_list = []
for col_name, dat_name in zip(column_names, data_name):
    zipped_list.append([col_name, dat_name])
zipped_list

[['Date', '32. Distribution Efficiency Data.csv'],
 ['State', '32. Distribution Efficiency Data.csv'],
 ['City', '32. Distribution Efficiency Data.csv'],
 ['Retailer', '32. Distribution Efficiency Data.csv'],
 ['Store Type', '32. Distribution Efficiency Data.csv'],
 ['Sales Value', '32. Distribution Efficiency Data.csv'],
 ['Units Sold', '32. Distribution Efficiency Data.csv'],
 ['Units in Inventory', '32. Distribution Efficiency Data.csv'],
 ['Supply Type', '32. Distribution Efficiency Data.csv'],
 ['Inventory Status', '32. Distribution Efficiency Data.csv'],
 ['Date', '6.MS_data_for_demo.csv'],
 ['Region', '6.MS_data_for_demo.csv'],
 ['Area', '6.MS_data_for_demo.csv'],
 ['Territory', '6.MS_data_for_demo.csv'],
 ['Team', '6.MS_data_for_demo.csv'],
 ['Market', '6.MS_data_for_demo.csv'],
 ['Company', '6.MS_data_for_demo.csv'],
 ['Brand', '6.MS_data_for_demo.csv'],
 ['Molecule', '6.MS_data_for_demo.csv'],
 ['MS%', '6.MS_data_for_demo.csv'],
 ['Sales', '6.MS_data_for_demo.csv'],
 ['Broad 

In [40]:
entity_data = pd.DataFrame(zipped_list, columns=["column_names", "data_source"])
entity_data["column_names"] = entity_data["column_names"].apply(lambda x: x.strip().lower())
entity_data.drop_duplicates(subset=["column_names"], keep="first", inplace=True)
entity_data.reset_index(drop=True, inplace=True)
entity_data

,column_names,data_source
0,date,32. Distribution Efficiency Data.csv
1,state,32. Distribution Efficiency Data.csv
2,city,32. Distribution Efficiency Data.csv
3,retailer,32. Distribution Efficiency Data.csv
4,store type,32. Distribution Efficiency Data.csv
...,...,...
196,hr name,17. Bank - Talent Acquisition.csv
197,hr id,17. Bank - Talent Acquisition.csv
198,recruitment source,17. Bank - Talent Acquisition.csv
199,dump period,17. Bank - Talent Acquisition.csv


In [37]:
entity_data.to_excel("../data/entity_data.xlsx", index=False)